In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
import pandas as pd
from datetime import datetime


In [0]:
dbutils.fs.ls("abfss://schemafolder@migrationstoragerinith.dfs.core.windows.net/first_method")

### Schema Evolution checking if any schema changes 

In [0]:
tables_metadata = [
    {"table_name": "orders", "path": "abfss://bronze@migrationstoragerinith.dfs.core.windows.net/first_method/orders"},
]


In [0]:
from pyspark.sql.types import StructType
import json

def validate_schema(table_name, df_new):
    schema_path = f"abfss://schemafolder@migrationstoragerinith.dfs.core.windows.net/first_method/{table_name}_schema.json"
    
    try:
        # Read existing schema JSON from ADLS
        baseline_json = (
            spark.read.text(schema_path)
            .collect()[0][0]
        )
        baseline_schema = StructType.fromJson(json.loads(baseline_json))
        
        # Compare columns
        baseline_cols = set([f.name for f in baseline_schema.fields])
        new_cols = set(df_new.columns)
        
        added_cols = new_cols - baseline_cols
        removed_cols = baseline_cols - new_cols

        if added_cols or removed_cols:
             Exception(f" Schema drift in `{table_name}`:\n Added: {added_cols}\n Removed: {removed_cols}")
        else:
            print(f"Schema matches for `{table_name}`")

    except Exception as e:
        print(f"⚠️ No baseline schema found or error occurred: {e}")
        print(f"🔁 Saving new schema for `{table_name}`")

        schema_json = df_new.schema.json()

        # Save as JSON string using Spark (correctly to ADLS)
        spark.createDataFrame([(schema_json,)], ["schema"]) \
            .write.mode("overwrite") \
            .text(schema_path)

In [0]:
for table in tables_metadata:
    table_name = table["table_name"]
    path = table["path"]

    try:
        df = spark.read.option("header", True).csv(path)
        validate_schema(table_name, df)
        print(f"Processed schema check for: {table_name}")
    except Exception as e:
        print(f"Failed for {table_name}: {str(e)}")


In [0]:
path="abfss://bronze@migrationstoragerinith.dfs.core.windows.net/first_method/orders"

df=spark.read \
    .format("csv") \
    .option("header",True) \
    .option("inferschema",True) \
    .load(path)


print(df.columns)
df.printSchema()

### Clean column names


In [0]:
import re

def clean_colnames(df):
    original_cols = df.columns
    for colname in original_cols:
        cleaned_name = re.sub(r"[^\w]", "_", colname.strip().lower())
        df = df.withColumnRenamed(colname, cleaned_name)
    return df

def clean_string_columns(df):
    for col_name, dtype in df.dtypes:
        if dtype == 'string':
            df = df.withColumn(col_name, regexp_replace(trim(lower(col(col_name))), r'[^a-zA-Z0-9\s]', ''))
    return df

def fill_missing(df, fill_dict):
    return df.fillna(fill_dict)

def remove_duplicates(df, subset_cols):
    return df.dropDuplicates(subset=subset_cols)


In [0]:
df = spark.read.option("header", True).csv("abfss://bronze@migrationstoragerinith.dfs.core.windows.net/first_method/orders")
df = clean_colnames(df)

# DROP if critical IDs are missing
df = df.dropna(subset=["order_id", "customer_id", "product_id"])

cast_columns = {
    "order_id": "int",
    "order_uuid": "string",
    "order_date": "date",
    "order_status": "string",
    "payment_method": "string",
    "store_id": "int",
    "year": "int",
    "month": "int",
    "month_name": "string",
    "week": "int",
    "day": "int",
    "day_of_week": "string",
    "day_name": "string",
    "is_weekend": "boolean",
    "customer_id": "int",
    "first_name": "string",
    "last_name": "string",
    "email": "string",
    "phone": "string",
    "address": "string",
    "city": "string",
    "state": "string",
    "zip_code": "int",
    "age": "int",                # Changed from float → int for better validation and consistency
    "gender": "string",
    "annual_income": "float",
    "customer_since": "date",
    "loyalty_tier": "string",
    "preferred_category": "string",
    "product_id": "int",
    "product_name": "string",
    "category": "string",
    "brand": "string",
    "sku": "string",
    "weight_lbs": "float",
    "dimensions": "string",
    "in_stock": "boolean",
    "quantity": "int",
    "unit_price": "float",
    "line_total": "float",
    "order_subtotal": "float",
    "discount_percent": "int",
    "discount_amount": "float",
    "tax_rate": "float",
    "tax_amount": "float",
    "shipping_cost": "float",
    "total_amount": "float"
}





for col_name, dtype in cast_columns.items():
    if col_name in df.columns:
        df = df.withColumn(col_name, col(col_name).cast(dtype))

# DATE fields
if "customer_since" in df.columns:
    df = df.withColumn("customer_since", to_date(col("customer_since"), "yyyy-MM-dd"))
if "date" in df.columns:
    df = df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

# GENDER Standardization
if "gender" in df.columns:
    gender_map = {'m': 'Male', 'male': 'Male', 'f': 'Female', 'female': 'Female'}
    df = df.withColumn("gender", lower(col("gender")))
    df = df.replace(gender_map, subset=["gender"])

# PAYMENT METHOD & ORDER STATUS Standardization
if "payment_method" in df.columns:
    payment_map = {
        "credit card": "Credit Card", "cash": "Cash", "paypal": "PayPal", 
        "gift card": "Gift Card", "debit card": "Debit Card", "debit": "Debit"
    }
    df = df.withColumn("payment_method", lower(trim(col("payment_method"))))
    df = df.replace(payment_map, subset=["payment_method"])

if "order_status" in df.columns:
    status_map = {
        "completed": "Completed", "shipped": "Shipped", "cancelled": "Cancelled",
        "returned": "Returned", "processing": "Processing"
    }
    df = df.withColumn("order_status", lower(trim(col("order_status"))))
    df = df.replace(status_map, subset=["order_status"])

# OUTLIERS
if "quantity" in df.columns:
    df = df.withColumn("quantity", when((col("quantity") <= 0) | (col("quantity") > 100), None).otherwise(col("quantity")))

if "price" in df.columns:
    df = df.withColumn("price", when(col("price") > 5000, 5000).otherwise(col("price")))

if "age" in df.columns:
    df = df.withColumn("age", when((col("age") < 18) | (col("age") > 90), None).otherwise(col("age")))

if "annual_income" in df.columns:
    df = df.withColumn("annual_income", when(col("annual_income") < 10000.0, 10000)
                                         .when(col("annual_income") > 500000.0, 500000)
                                         .otherwise(col("annual_income")))

# NULL FILLING
fill_values = {
    "phone": "Not Provided",
    "customer_id": -1,
    "product_id": -1
}

if "phone" in df.columns:
    fill_values["phone"] = "Not Provided"
if "brand" in df.columns:
    fill_values["brand"] = "Unknown"
df = fill_missing(df, fill_values)

# REMOVE DUPLICATES
df = clean_string_columns(df)



In [0]:
dim_customer = df.select(
    "customer_id",
    "first_name",
    "gender",
    "age",
    "annual_income",
    "customer_since",
    "loyalty_tier",
    "address",
    "city",
    "state",
    "zip_code",
    "phone"
).dropDuplicates(["customer_id"])



In [0]:
dim_product = df.select(
    "product_id",
    "product_name",
    "category",        # previously might've been "product_category"
    "unit_price"       # renamed from "product_price"
).dropDuplicates(["product_id"])


In [0]:
dim_date = df.select("order_date").dropDuplicates() \
    .withColumn("date_key", to_date(col("order_date"), "yyyy-MM-dd")) \
    .withColumn("year", year(col("date_key"))) \
    .withColumn("month", month(col("date_key"))) \
    .withColumn("day", dayofmonth(col("date_key"))) \
    .withColumn("day_of_week", dayofweek(col("date_key"))) \
    .withColumn("quarter", quarter(col("date_key")))


In [0]:
fact_orders = df.select(
    "order_id",
    "customer_id",
    "product_id",
    "order_date",
    "quantity",
    "line_total",
    "discount_percent",
    "tax_rate",
    "shipping_cost",
    "total_amount",
    "payment_method"
)


In [0]:

dim_customer.write \
            .format("delta") \
            .mode("overwrite") \
            .save("abfss://silver@migrationstoragerinith.dfs.core.windows.net/first_method/customers/")


In [0]:
fact_orders.write.format("delta").mode("overwrite").save("abfss://silver@migrationstoragerinith.dfs.core.windows.net/first_method/orders/")


In [0]:
dim_date.write.format("delta").mode("overwrite").save("abfss://silver@migrationstoragerinith.dfs.core.windows.net/first_method/date/")


In [0]:
dim_product.write.format("delta").mode("overwrite").save("abfss://silver@migrationstoragerinith.dfs.core.windows.net/first_method/products/")


### Moving file into archive

In [0]:
current_date = datetime.now().strftime("%Y%m%d")

raw_file_path = "abfss://bronze@migrationstoragerinith.dfs.core.windows.net/first_method/orders/orders.csv"
archived_path = f"abfss://bronze@migrationstoragerinith.dfs.core.windows.net/first_method/archive/orders_{current_date}.csv"

dbutils.fs.mv(raw_file_path, archived_path)
